- Every object has an *identity*, *value*, and *type*.  
    - **Identity**: never changes after the object is created (it's similar to the object's memory address).  
        - The `is` operator compares object identities, and the `id()` function returns an integer representing the object's identity (in CPython, it returns the memory address of the object).  
    - **Type**: determines the operations an object supports and defines all possible values for that object.  
        - Python’s type system: *Duck Typing* (“If it looks like a duck and quacks like a duck, it’s a duck”).  
            - With **nominal typing**, an object belongs to a given type only if it is declared as such (or through mechanisms like class inheritance). With **duck typing**, an object is considered to be of a given type if it has all the methods and properties required by that type — therefore, no explicit type checking is needed.  
        - Type is immutable.  
        - The `type()` function returns the object’s type, which is itself an object.  
    - The *value* of an object may change.
- In Python, everything is an object — including classes: a class is itself an instance of the built-in `type` class, which is the class of classes (including itself).
- `type` is the default metaclass.  
    - The metaclass controls how Python builds a class internally, how Python constructs the MRO (Method Resolution Order), how multiple inheritance works, etc.  
- Classes in Python are created at runtime: an object representing **a class** (the *class object*) is created **at the moment the class statement is executed by Python**, not when you instantiate the class.  
- For every class defined in Python, there is exactly **one** corresponding class object.  
- Classes are callable objects (they implement the `__call__()` method) that act as factories: they create and return an instance of themselves.
- Steps for classes and instances creation
    1. Class creation
        
        ```python
        class A(B):
            x = 10
            def __init__(self, v):
                self.v = v
        ```
        
        1. Namespace creation:
            
            ```python
            {"x": 10, "__init__": <function ...>}
            ```
            
        2. Python collects the class name (`A`), the base classes (`B`), and the constructed namespace.
        3. It calls the metaclass (normally `type`):
            
            ```python
            A = type("A", (B,), {"x": 10, "__init__": ...})
            ```
            
            1. `type.__new__` creates the class object.
            2. `type.__init__` initializes the class object.
            3. Result: `A` is an object (an instance of `type`).
    2. Instance creation
        
        ```python
        obj = A(5)
        ```
        
        1. `A(5)` triggers `A.__call__(5)`.
        2. Inside this method:
            1. Execution of `A.__new__`
                
                ```python
                instance = A.__new__(A, 5) # A must be passed because __new__ is static
                ```
                
                - Allocates the object on the heap and returns the newly created (but not yet initialized) instance.
                - If `__new__` returns an object of class `A` itself, then step b is executed.
            2. Execution of `A.__init__`
                
                ```python
                instance.__init__(5) # equivalent to MyClass.__init__(instance, 5)
                ```
                
                Returns the created instance.
- `super()`: a function that allows accessing **superclass** methods without creating new instances — it does not allocate memory and does not call a constructor.
    - The **super** of a class is not always the “direct parent class,” but rather **the next class in the MRO sequence.**
- About `object.__new__(cls, ...)`:
    - Creates a new instance of class `cls`.
    - **Special static method** (no need to declare it as `@staticmethod`).
    - First argument → the **class** from which you want to create an instance (`cls`).
    - The remaining arguments → the same ones you passed when calling the class (`MyClass(arg1, arg2)`).
        - Even though it does not initialize the instance, it still needs access to these arguments. It’s also useful for debugging. That’s why they are passed.
    - If `__new__()` returns an instance of `cls`, Python will call `__init__()`; otherwise, it will not be called.
- About an **object's namespace**
    - It is simply a **dictionary where the object's attributes/methods are stored**.
    - It can be accessed via `obj.__dict__`.
- Note: class attributes **do not become** instance attributes (they will not appear in the instance’s `__dict__`). They **still belong to the class**, but **can be accessed by the instance** because of the *attribute lookup* mechanism (**if the attribute is not in the instance**, Python looks in the class—and then in the parent classes).
    - It is not possible to modify a class attribute from the instance: if an assignment is made through the instance, Python creates (or overrides) an attribute only on the instance (a new field appears on the instance, completely independent of the class field).
        
        ```python
        class A:
            x = 10
        
        a = A()
        b = A()
        
        a.x = 20
        
        print(a.x)  # 20  (from the instance)
        print(b.x)  # 10  (from the class)
        print(A.x)  # 10  (from the class)
        
        print(a.__dict__) # {'x': 20}
        print(b.__dict__) # {}
        ```
        
- `__dict__`
    - It is an **internal dictionary** that stores the “writable” (modifiable) attributes of an object.
    - `__dict__` in **instances**
        - The instance **stores its own attributes here**.
        - Attributes created on the instance **are unrelated to class attributes**.
    - `__dict__` in a **class**
        - Class attributes and methods
        - Properties and internal metadata
    - When searching for an attribute or method, Python follows this order:
        - Look in `a.__dict__`
        - If not found, look in `A.__dict__`
        - If not found, look in the parent classes
        - If not found, raise `AttributeError`
    - Instances **do not copy** class attributes. They **inherit access**, but do not store a copy.
    - If a class uses `__slots__`, instances will not have `__dict__` (see Python docs on slots).
- **`object`** is **the base class of *all* instances** in Python.
    - Everything in Python is an object because everything inherits from `object` — it is the last class in the MRO.
- **`type`** is **the base class of *all* classes**.
    - It is the **default metaclass** in Python.
- `object` is an instance of `type`.
- `type` is a child of `object`.
- This is an intentional and well-designed loop.


In [1]:
print("===> About object class:")
print(object.__class__)
print(object.__bases__) # __bases__ gives a tuple of base classes
print(" ".join(object.__dict__.keys()))
print("")
print("===> About type class:")
print(type.__class__)
print(type.__bases__)
print(" ".join(type.__dict__.keys()))

===> About object class:
<class 'type'>
()
__new__ __repr__ __hash__ __str__ __getattribute__ __setattr__ __delattr__ __lt__ __le__ __eq__ __ne__ __gt__ __ge__ __init__ __reduce_ex__ __reduce__ __getstate__ __subclasshook__ __init_subclass__ __format__ __sizeof__ __dir__ __class__ __doc__

===> About type class:
<class 'type'>
(<class 'object'>,)
__new__ __repr__ __call__ __getattribute__ __setattr__ __delattr__ __init__ __or__ __ror__ mro __subclasses__ __prepare__ __instancecheck__ __subclasscheck__ __dir__ __sizeof__ __basicsize__ __itemsize__ __flags__ __weakrefoffset__ __base__ __dictoffset__ __name__ __qualname__ __bases__ __mro__ __module__ __abstractmethods__ __dict__ __doc__ __text_signature__ __annotations__ __type_params__


In [2]:
def say_hello(self):
    print("Hello, World! My name is", self.name)

Example1 = type("Example1", (object,), {"say_hello": say_hello, "name": "ExampleInstance"}) # Create a new class 'Example1' with a method 'say_hello' and an attribute 'name'
print(Example1.__dict__)  # Access the 'name' attribute

# Equivalent to:
# class Example1:
#     name = "ExampleInstance"

#     def say_hello(self):
#         print("Hello, World! My name is", self.name)

{'say_hello': <function say_hello at 0x106496480>, 'name': 'ExampleInstance', '__module__': '__main__', '__dict__': <attribute '__dict__' of 'Example1' objects>, '__weakref__': <attribute '__weakref__' of 'Example1' objects>, '__doc__': None}


In [3]:
ex1 = Example1()  # Instantiate the class
ex1.say_hello()  # Call the method
ex1.name = "NewName"  # Modify the attribute
ex1.say_hello()  # Call the method again to see the updated name
print(type(ex1))  # Check the type of the instance

Hello, World! My name is ExampleInstance
Hello, World! My name is NewName
<class '__main__.Example1'>


In [4]:
class Example2:
    class_attribute = "I am a class attribute"

    def __new__(cls, *args, **kwargs):
        print("Calling __new__ method")
        return super().__new__(cls)

    def __init__(self, instance_attribute):
        self.instance_attribute = instance_attribute
    
    def new_attribute_creation(self, value):
        self.new_attribute = value

print(Example2.__dict__)  # Access class attributes and methods
print(Example2.__mro__)  # Method Resolution Order

{'__module__': '__main__', '__firstlineno__': 1, 'class_attribute': 'I am a class attribute', '__new__': <staticmethod(<function Example2.__new__ at 0x106496700>)>, '__init__': <function Example2.__init__ at 0x106496200>, 'new_attribute_creation': <function Example2.new_attribute_creation at 0x1064967a0>, '__static_attributes__': ('instance_attribute', 'new_attribute'), '__dict__': <attribute '__dict__' of 'Example2' objects>, '__weakref__': <attribute '__weakref__' of 'Example2' objects>, '__doc__': None}
(<class '__main__.Example2'>, <class 'object'>)


In [5]:
ex2 = Example2.__new__(Example2)
ex2.__init__("I am an instance attribute")

# Equivalent to:
# ex2 = Example2("I am an instance attribute")

Calling __new__ method


In [6]:
print(ex2.__dict__)  # Access instance attributes
ex2.new_attribute_creation("I am a new attribute")
print(ex2.__dict__)  # Access instance attributes after adding a new one
print(Example2.__dict__)  # Class attributes remain unchanged

{'instance_attribute': 'I am an instance attribute'}
{'instance_attribute': 'I am an instance attribute', 'new_attribute': 'I am a new attribute'}
{'__module__': '__main__', '__firstlineno__': 1, 'class_attribute': 'I am a class attribute', '__new__': <staticmethod(<function Example2.__new__ at 0x106496700>)>, '__init__': <function Example2.__init__ at 0x106496200>, 'new_attribute_creation': <function Example2.new_attribute_creation at 0x1064967a0>, '__static_attributes__': ('instance_attribute', 'new_attribute'), '__dict__': <attribute '__dict__' of 'Example2' objects>, '__weakref__': <attribute '__weakref__' of 'Example2' objects>, '__doc__': None}


In [7]:
ex2.class_attribute = "Modified class attribute in instance"
print(ex2.__dict__)  # Instance attributes after modifying class attribute
print(Example2.__dict__)  # Class attributes remain unchanged

# ex2.class_attribute will not modify the class attribute, it creates an instance attribute with the same name, leaving the class attribute intact.
# ex2.class_attribute now refers to the instance attribute, while Example2.class_attribute still refers to the original class attribute.

{'instance_attribute': 'I am an instance attribute', 'new_attribute': 'I am a new attribute', 'class_attribute': 'Modified class attribute in instance'}
{'__module__': '__main__', '__firstlineno__': 1, 'class_attribute': 'I am a class attribute', '__new__': <staticmethod(<function Example2.__new__ at 0x106496700>)>, '__init__': <function Example2.__init__ at 0x106496200>, 'new_attribute_creation': <function Example2.new_attribute_creation at 0x1064967a0>, '__static_attributes__': ('instance_attribute', 'new_attribute'), '__dict__': <attribute '__dict__' of 'Example2' objects>, '__weakref__': <attribute '__weakref__' of 'Example2' objects>, '__doc__': None}


In [8]:
def say_hello():
    print("Hello, World!")

ex2.say_hello = say_hello # Dynamically add method to instance
print(ex2.__dict__)  # Instance attributes before calling the new method
ex2.say_hello()  # Call the dynamically added method

{'instance_attribute': 'I am an instance attribute', 'new_attribute': 'I am a new attribute', 'class_attribute': 'Modified class attribute in instance', 'say_hello': <function say_hello at 0x1064960c0>}
Hello, World!


In [9]:
print(Example2.__dict__)  # Class attributes remain unchanged
Example2.new_attribute_creation(Example2, "Class level new attribute")
print(Example2.__dict__)  # Class attributes after adding a new one

{'__module__': '__main__', '__firstlineno__': 1, 'class_attribute': 'I am a class attribute', '__new__': <staticmethod(<function Example2.__new__ at 0x106496700>)>, '__init__': <function Example2.__init__ at 0x106496200>, 'new_attribute_creation': <function Example2.new_attribute_creation at 0x1064967a0>, '__static_attributes__': ('instance_attribute', 'new_attribute'), '__dict__': <attribute '__dict__' of 'Example2' objects>, '__weakref__': <attribute '__weakref__' of 'Example2' objects>, '__doc__': None}
{'__module__': '__main__', '__firstlineno__': 1, 'class_attribute': 'I am a class attribute', '__new__': <staticmethod(<function Example2.__new__ at 0x106496700>)>, '__init__': <function Example2.__init__ at 0x106496200>, 'new_attribute_creation': <function Example2.new_attribute_creation at 0x1064967a0>, '__static_attributes__': ('instance_attribute', 'new_attribute'), '__dict__': <attribute '__dict__' of 'Example2' objects>, '__weakref__': <attribute '__weakref__' of 'Example2' obj

- **Singleton**
    - A design pattern whose goal is to ensure that only one instance of a class exists throughout the entire program — and that everyone who needs it receives exactly the same object.
    - The most common mechanism is overriding `__new__`:
        
        ```python
        class Singleton:
            _inst = None
        
            def __new__(cls, *args, **kwargs):
                if cls._inst is None:
                    cls._inst = super().__new__(cls)
                return cls._inst
        ```
        
    - `super().__new__` inside this class is calling the `__new__` of the highest base class in the MRO — which is `object`, since `Singleton` does not inherit from any class other than `object`.

In [10]:
class Singleton:
    _instance = None

    def __new__(cls, *args, **kwargs):
        if not cls._instance:
            cls._instance = super().__new__(cls)
        return cls._instance
    
    def __init__(self, value):
        self.value = value

singleton1 = Singleton(1)
print(singleton1.value)
singleton2 = Singleton(2)
print(singleton1.value)
print(singleton2.value)

1
2
2


In [11]:
print(singleton1 is singleton2)  # This should print True, indicating both are the same instance

True
